# KNN User-User Collaborative Filtering

The idea is to find similar users based on their preferences or behavior and recommend items that similar users liked but the target user has not interacted with. If User A and User B have rated a lot of the same items similarly, User B might enjoy items that User A has rated highly but hasn’t yet discovered.

Based on [Introduction to Recommender System](https://towardsdatascience.com/introduction-to-recommender-systems-6c66cf15ada):

> Collaborative methods for recommender systems are methods that are **based solely on the past interactions** recorded between users and items in order to produce new recommendations. These interactions are stored in the so-called “user-item interactions matrix”. Then, the main idea that rules collaborative methods is that these past user-item interactions are sufficient to detect similar users and/or similar items and make predictions based on these estimated proximities.

> User-User CF is **applying KNN to find the neighbor users** with similar prior item preferences, and prediction of target item I for user U will be computed from the preferences of the neighbors.
Notice that, when computing similarity between users, the number of “common interactions” (how much items have already been considered by both users?) should be considered carefully! Indeed, most of the time, we want to avoid that someone that only have one interaction in common with our reference user could have a 100% match and be considered as being “closer” than someone having 100 common interactions and agreeing “only” on 98% of them. So, we consider that two users are similar if they have interacted with a lot of common items in the same way (similar rating, similar time hovering…).

In this notebook, we would take a very simple and straight-forward approach to implement KNN User-User CF. We formulate the problem as predicting the rating between user U and item I based on the rating records.

Specifically, we will:
- Collect input containing user-item rating
- Build user-user similarity matrix
- Measure similarities between users by using plain item rating vectors
- Predict the rating between user U and item I by getting all users who have rated I, identify N neighbors and weighted average their ratings based on how similar the users are to the user U

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import mlflow
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel

load_dotenv()

sys.path.insert(0, "..")

from src.eval import (
    create_label_df,
    create_rec_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.id_mapper import IDMapper
from src.math_utils import sigmoid
from src.model import User2UserCollaborativeFiltering
from src.train_utils import map_indice
from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L5 - Reco Algo"
    run_name: str = "002-cf-u2u"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-22 18:42:06.453 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L5 - Reco Algo - run 002-cf-u2u...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L5 - Reco Algo",
  "run_name": "002-cf-u2u",
  "notebook_persist_dp": "/Users/dvq/frostmourne/reco-algo/notebooks/data/002-cf-u2u",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Implement

In [4]:
def init_model(n_users, n_items):
    model = User2UserCollaborativeFiltering(n_users, n_items)
    return model

# Test implementation

In [5]:
# Mock data
user_indices = [0, 0, 1, 1, 2, 2, 2]
item_indices = [0, 1, 1, 2, 3, 1, 2]
ratings = [1, 4, 4, 5, 3, 2, 4]
n_users = len(set(user_indices))
n_items = len(set(item_indices))

val_user_indices = [0, 1, 2]
val_item_indices = [2, 1, 2]
val_ratings = [2, 4, 5]

print("Mock User IDs:", user_indices)
print("Mock Item IDs:", item_indices)
print("Ratings:", ratings)

model = init_model(n_users, n_items)

users = [0, 1, 2]
items = [2, 2, 0]
predictions = model.predict(users, items)
print(predictions)

Mock User IDs: [0, 0, 1, 1, 2, 2, 2]
Mock Item IDs: [0, 1, 1, 2, 3, 1, 2]
Ratings: [1, 4, 4, 5, 3, 2, 4]
[0.5 0.5 0.5]


In [6]:
model.fit(user_indices, item_indices, ratings)
predictions = model.predict(users, items)
print(predictions)

[0.99031217 0.98201379 0.73105858]


#### 🧐 Go into details

In [7]:
model.user_item_matrix

array([[1., 4., 0., 0.],
       [0., 4., 5., 0.],
       [0., 2., 4., 3.]])

In [8]:
model.user_similarity

array([[0.        , 0.60604322, 0.36030188],
       [0.60604322, 0.        , 0.81202071],
       [0.36030188, 0.81202071, 0.        ]])

In [9]:
# Inspect one example
user = 0
item = 2

sim_scores = model.user_similarity[user]
print(f"{sim_scores=}")

sim_scores=array([0.        , 0.60604322, 0.36030188])


In [10]:
# Only consider users who have rated the item
user_ratings = model.user_item_matrix[:, item]
print(f"{user_ratings=}")
sim_scores = sim_scores[user_ratings != 0]
print(f"{sim_scores=}")
user_ratings = user_ratings[user_ratings != 0]
print(f"{user_ratings=}")

user_ratings=array([0., 5., 4.])
sim_scores=array([0.60604322, 0.36030188])
user_ratings=array([5., 4.])


In [11]:
# Weighted average of ratings
print(f"Weighted average: {np.dot(sim_scores, user_ratings)}")
print(f"Normalization factor: {np.sum(sim_scores)}")
print(f"Predicted rating: {np.dot(sim_scores, user_ratings) / np.sum(sim_scores)}")
print(
    f"Predicted rating - sigmoid: {sigmoid(np.dot(sim_scores, user_ratings) / np.sum(sim_scores))}"
)

Weighted average: 4.471423593469625
Normalization factor: 0.9663450945516922
Predicted rating: 4.627149885356445
Predicted rating - sigmoid: 0.9903121712214553


In [12]:
recommendations = model.recommend(val_user_indices, k=2)

Generating recommendations:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
recommendations

{'user_indice': [np.int64(0),
  np.int64(0),
  np.int64(1),
  np.int64(1),
  np.int64(2),
  np.int64(2)],
 'recommendation': [np.int64(2),
  np.int64(1),
  np.int64(2),
  np.int64(3),
  np.int64(2),
  np.int64(1)],
 'score': [np.float64(0.9903121712214553),
  np.float64(0.9628273118576526),
  np.float64(0.9820137900379085),
  np.float64(0.9525741268224334),
  np.float64(0.9933071490757153),
  np.float64(0.9820137900379085)]}

# Prep data

In [14]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm = IDMapper().load("../data/idm.json")
assert (val_df[args.timestamp_col].min() - train_df[args.timestamp_col].max()) > 0
val_timestamp = train_df[args.timestamp_col].max() + 1
print(f"{val_timestamp=}")

val_timestamp=np.int64(1628641464793)


In [15]:
user_ids = train_df[args.user_col].values
item_ids = train_df[args.item_col].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))
n_users = len(unique_user_ids)
n_items = len(unique_item_ids)

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-09-22 18:42:07.837 | INFO     | __main__:<module>:8 - len(unique_user_ids)=20,366, len(unique_item_ids)=4,696


In [16]:
train_df = train_df.pipe(map_indice, idm, args.user_col, args.item_col)
val_df = val_df.pipe(map_indice, idm, args.user_col, args.item_col)

user_indices = [idm.get_user_index(user_id) for user_id in user_ids]
item_indices = [idm.get_item_index(item_id) for item_id in item_ids]
ratings = train_df[args.rating_col].values.tolist()

val_user_indices = [idm.get_user_index(user_id) for user_id in val_df[args.user_col]]
val_item_indices = [idm.get_item_index(item_id) for item_id in val_df[args.item_col]]
val_ratings = val_df[args.rating_col].values.tolist()

# Train

In [17]:
model = init_model(n_users, n_items)

#### Predict before train

In [18]:
user_id = val_df.sample(1)[args.user_col].values[0]
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
test_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence
129,AEQ7HJ3JNH63S4B3CGQNOJ2LYUQA,B001EYUS6M,0.0,1639091854918,17119,505,Video Games,Fantastic Four: Rise of the Silver Surfer,[The Fantastic Four confront their greatest ad...,"[Video Games, Legacy Systems, Nintendo Systems...",23.96,"[-1, -1, -1, 3455, 4441, 653, 3599, 2270, 2533..."
767,AEQ7HJ3JNH63S4B3CGQNOJ2LYUQA,B0B96RSG2Y,5.0,1639091417894,17119,4489,Computers,Razer Ornata V3 Gaming Keyboard: Low-Profile K...,[Meet the Razer Ornata V3—a low-profile ergono...,"[Video Games, PC, Accessories, Gaming Keyboards]",59.99,"[-1, -1, -1, -1, 3455, 4441, 653, 3599, 2270, ..."
886,AEQ7HJ3JNH63S4B3CGQNOJ2LYUQA,B001ECIU96,0.0,1639091417894,17119,4475,Video Games,Rayman Raving Rabbids 2 - Nintendo DS,[The mischievous bunnies are back in a hilario...,"[Video Games, Legacy Systems, Nintendo Systems...",21.99,"[-1, -1, -1, -1, 3455, 4441, 653, 3599, 2270, ..."
1615,AEQ7HJ3JNH63S4B3CGQNOJ2LYUQA,B09M6V916D,5.0,1639091854918,17119,3538,Computers,Razer Basilisk V2 Wired Gaming Mouse + Mouse B...,[],"[Video Games, PC, Accessories, Gaming Mice]",None,"[-1, -1, -1, 3455, 4441, 653, 3599, 2270, 2533..."


In [19]:
item_id = test_df.loc[lambda df: df[args.rating_col].gt(0)][args.item_col].values[0]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
user_indice = idm.get_user_index(user_id)
item_indice = idm.get_item_index(item_id)

model.predict([user_indice], [item_indice])

2024-09-22 18:42:08.086 | INFO     | __main__:<module>:2 - Test predicting before training with user_id = AEQ7HJ3JNH63S4B3CGQNOJ2LYUQA and parent_asin = B0B96RSG2Y


array([0.5])

#### Training loop

In [20]:
model.fit(user_indices, item_indices, ratings)

# Predict

In [21]:
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
model.predict([user_indice], [item_indice])

2024-09-22 18:42:15.337 | INFO     | __main__:<module>:1 - Test predicting before training with user_id = AEQ7HJ3JNH63S4B3CGQNOJ2LYUQA and parent_asin = B0B96RSG2Y


array([0.99330715])

# Evaluate

## Ranking metrics

In [22]:
recommendations = model.recommend(val_user_indices, k=args.top_K)

Generating recommendations:   0%|          | 0/1898 [00:00<?, ?it/s]

In [23]:
recommendations_df = pd.DataFrame(recommendations).pipe(create_rec_df, idm)
recommendations_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,11638,1187,0.993307,1.0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B001BZ2F56
1,11638,3369,0.993307,2.0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B087STK7JZ
2,11638,1355,0.993307,3.0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B07BZP7HML
3,11638,3350,0.993307,4.0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B005OGL766
4,11638,1351,0.993307,5.0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B003O6ED7S
...,...,...,...,...,...,...
189795,10038,2511,0.993307,196.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B08VS3Z468
189796,10038,2519,0.993307,197.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B019GXXRRO
189797,10038,2520,0.993307,198.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B0128UH1VU
189798,10038,2521,0.993307,199.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B001ELJFNW


In [24]:
label_df = create_label_df(val_df)
label_df

,user_id,parent_asin,rating,rating_rank
548,AFI3SEGKSS7X7CVOC3HS5U5RAIWQ,B077GG9D5D,5.0,1.0
697,AGDFPA6XVENIY7XJPIOXBUHRJQ6A,B09V25XG1G,5.0,1.0
109,AHMJVCKVHJIT2R5NWWV4HG4TDH6A,B07C2XYDW8,5.0,1.0
197,AGQAPJGNR3IDAJCPD2YPU7CRRPBA,B0C7BN9G35,5.0,1.0
797,AF6IUPCJM4FDNOQZNCDOGTLM2M4Q,B09T5VN7D1,4.0,1.0
...,...,...,...,...
850,AFB6FYPPCN33UMUU5536IHXNOHCQ,B001E8WQJA,0.0,18.0
1106,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B00DBLBMBQ,0.0,18.0
1003,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B00Z9TMBOU,0.0,18.0
52,AFB6FYPPCN33UMUU5536IHXNOHCQ,B0892HRCST,0.0,19.0


In [25]:
eval_df = merge_recs_with_target(recommendations_df, label_df, k=args.top_K)
eval_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
196,14192.0,4695.0,0.993307,1,AE2AZ2MNROPF33U6SS53VI22OXJA,B09S95BGBV,0,NaN
70,14192.0,3160.0,0.993307,2,AE2AZ2MNROPF33U6SS53VI22OXJA,B004K6LQ9E,0,NaN
22,14192.0,3126.0,0.993307,3,AE2AZ2MNROPF33U6SS53VI22OXJA,B000G7X5MM,0,NaN
120,14192.0,3127.0,0.993307,4,AE2AZ2MNROPF33U6SS53VI22OXJA,B00NY7H5EA,0,NaN
108,14192.0,1094.0,0.993307,5,AE2AZ2MNROPF33U6SS53VI22OXJA,B00GOOSORM,0,NaN
...,...,...,...,...,...,...,...,...
191525,16101.0,549.0,0.993307,196,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B008J35YFQ,0,NaN
191476,16101.0,3769.0,0.993307,197,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B000A15K0Y,0,NaN
191595,16101.0,924.0,0.993307,198,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B01MXE0FKC,0,NaN
191496,16101.0,3765.0,0.993307,199,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B001TERJD2,0,NaN


In [26]:
ranking_report = log_ranking_metrics(args, eval_df)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:61: RuntimeWarning: invalid value encountered in divide
  return (1 + beta_sqr) * precision_arr * recall_arr / (beta_sqr * precision_arr + recall_arr)


## Classification metrics

In [27]:
val_user_indices = val_df["user_indice"].values
val_item_indices = val_df["item_indice"].values

In [28]:
classifications = model.predict(val_user_indices, val_item_indices)

In [29]:
eval_classification_df = val_df.assign(
    classification_proba=classifications,
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,classification_proba,label
0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B01GY35QPU,0.0,1638057548682,11638,1902,Video Games,Detroit Become Human - PlayStation 4,[Detroit: Become Human is the latest title in ...,"[Video Games, PlayStation 4, Games]",21.4,"[3579, 878, 434, 1579, 3669, 3322, 3495, 2339,...",0.500000,0
1,AEMYS2WV33NDV3OAJFOM7SSSIDTA,B0088TN7BO,0.0,1640914537652,14788,2180,Video Games,LEGO Lord of the Rings - Nintendo 3DS,"[Product Description, Based on The Lord of the...","[Video Games, Legacy Systems, Nintendo Systems...",19.95,"[-1, -1, -1, -1, -1, 3244, 2872, 2621, 3048, 3...",0.500000,0
2,AEXKGQQMYQQUNWVGD66TG3VT4V4A,B00IPTUJ8G,0.0,1653120417877,19789,4619,Video Games,Borderlands 2,"[Borderlands 2 for PS Vita, View Larger, View ...","[Video Games, Legacy Systems, PlayStation Syst...",48.1,"[-1, -1, -1, -1, -1, 2358, 3142, 4521, 2292, 2...",0.978948,0
3,AGQAPJGNR3IDAJCPD2YPU7CRRPBA,B00CMQTVK0,0.0,1643392366551,6414,1765,Video Games,Xbox One with Kinect (Day One Edition),"[Get more with Xbox One., Introducing Xbox One...","[Video Games, Xbox One, Consoles]",589.99,"[-1, -1, -1, -1, 3142, 4274, 4345, 809, 416, 1...",0.978117,0
4,AGIJWTPKBANKWNEM2AHK7PWTBYLQ,B01BF9X9VQ,0.0,1647513889874,1195,2412,Video Games,World of Warcraft: Legion - Standard Edition -...,"[Kingdoms will burn, The Burning Legion surges...","[Video Games, PC, Games]",28.6,"[170, 3609, 1588, 1467, 3786, 1627, 4234, 3056...",0.989117,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AF5T2J7T33UFSPUGCKTPTGU7EY7A,B008I2LRMC,0.0,1654892101229,14749,1743,Video Games,F1: 2012,"[Product Description, F1 2012 is designed to b...","[Video Games, Legacy Systems, Xbox Systems, Xb...",33.03,"[601, 1842, 3391, 2640, 761, 3624, 1079, 2656,...",0.500000,0
1894,AGXQLR7TWHVUQLDBKKOBI4OQQGRQ,B01N3ASPNV,0.0,1637285457188,8208,3069,All Electronics,amFilm Tempered Glass Screen Protector for Nin...,[],"[Video Games, Nintendo Switch, Accessories, Fa...",8.91,"[-1, -1, 4227, 1039, 1352, 2619, 2655, 1305, 1...",0.993307,0
1895,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,9221,2581,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 2705, 3691, 1804, 111, 3330, 1775, 2979, ...",0.731059,1
1896,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,4173,3191,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 2379, 1799, 1932, 1568, 161]",0.980681,1


In [30]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# Clean up

In [31]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/22 18:51:08 INFO mlflow.tracking._tracking_service.client: 🏃 View run 002-cf-u2u at: http://localhost:5003/#/experiments/1/runs/b10260770f8241eba7a1581ca812dfc7.
2024/09/22 18:51:08 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/1.


# Appendix

## Model returning same score for every user-item in top 100